<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf_function_and_auto_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0-rc1
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 0.25.3
sklearn 0.22.2.post1
tensorflow 2.2.0-rc1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [0]:
# tf.function and auto-graph.
# 下面演示tf.function的使用
def scaled_elu(z, scale=1.0, alpha=1.0):
    # 这个函数做的是
    # z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z)) #tensorflow里使用where实现三元表达式
    # 我们在这个函数里面使用tensorflow里面定义函数的原因是，如果我们使用python定义的函数，
    # 当输入是向量的时候，不能做到对向量里面的每个值都进行操作
    # 所以这个函数使用了tf内置的一些函数，当然了这个函数的整体实现呢，还是使用了python语法
    # 所以说这还是一个python的函数

# 我们用一下这个函数
print(scaled_elu(tf.constant(-3.))) # 输入一个标量，就对这个值做操作
print(scaled_elu(tf.constant([-3., -2.5]))) #输入一个向量,就会对向量里的每个值都进行操作
# 输出上面的结果我们可以看到

#现在我们用tf.function 把这个python函数scaled_elu呢，转化为tensorflow里图实现的函数
#我们使用tf.function，把这个函数名传进去就可以了
#非常简单，我只需要一行代码，就可以把python实现的代码，转化为tensorflow里图实现的函数
scaled_elu_tf = tf.function(scaled_elu) 

#测下这个函数
print(scaled_elu_tf(tf.constant(-3.))) #输入标量
print(scaled_elu_tf(tf.constant([-3., -2.5]))) #输入向量
#我们可以看出跟上面的结果是一样的

#对于转化后的函数呢，我们可以通过scaled_elu_tf.python_function来找回它的python函数有
print(scaled_elu_tf.python_function is scaled_elu)

#我们转化为图结构以后呢，有什么优势
#快， tensorflow图结构里面有很多的优化，拿一个普通的python函数转为tensorflow图结构函数以后呢，
#    速度上会有一些优势


tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
True


In [0]:
%timeit scaled_elu(tf.random.normal((1000, 1000))) #我们测一下普通的函数
%timeit scaled_elu_tf(tf.random.normal((1000, 1000))) #我们使用图实现的函数
#如果有GPU的话，它的加速比会更大

10 loops, best of 3: 26.5 ms per loop
10 loops, best of 3: 24.2 ms per loop


In [0]:
# 上面呢，我们使用了tf.function来显式的把python函数转化为tensorflow里面的图
# 那么有没有别的方式呢，就是下面的方法，可以除了tf.function直接调用之外呢，
# 可以去用它的annotation注解去把一个function转成图

# 1 + 1/2 + 1/2^2 + ... + 1/2^n
@tf.function # 如何把converge_to_2这个函数变成tensorflow里面的图呢，很简单，在最顶上加上@tf.function，加上一个标注，用这种方式也可以把普通函数转为tensorflow里面的图
def converge_to_2(n_iters): # 这里定义一个函数叫converge_to_2，参数是迭代多少次，也就是n的大小，
    # 为什么函数名叫converge_to_2呢，因为1 + 1/2 + 1/2^2 + ... + 1/2^n 极限为2
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(20)) #调用函数

tf.Tensor(1.9999981, shape=(), dtype=float32)


In [0]:
def display_tf_code(func):
    # 还有一个方法是可以把这个python函数转为可以生成tensorflow图结构的中间代码给大家展示出来
    # 这个tensorflow代码呢就可以生成tensorflow里面的图结构
    # 传入的是一个python的函数
    # 这个函数可以把普通的python代码转成tensorflow代码，这个tensorflow代码呢就可以生成tensorflow里面的图结构
    code = tf.autograph.to_code(func) #把func传进去 可以得到一段源代码 我们在notebook里还可以很方便的把源代码展示出来
    from IPython.display import display, Markdown #这时候就需要去调用IPython里面的display函数和markdown库
    display(Markdown('```python\n{}\n```'.format(code))) #然后我们调用display函数，把code用markdown的语法写出来

In [0]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
    do_return = False
    retval_ = ag__.UndefinedReturnValue()
    with ag__.FunctionScope('scaled_elu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        is_positive = ag__.converted_call(tf.greater_equal, (z, 0.0), None, fscope)
        try:
            do_return = True
            retval_ = fscope.mark_return_value((scale * ag__.converted_call(tf.where, (is_positive, z, (alpha * ag__.converted_call(tf.nn.elu, (z,), None, fscope))), None, fscope)))
        except:
            do_return = False
            raise
    (do_return,)
    return ag__.retval(retval_)

```

In [0]:
# 在之前的例子中，我们都是把变量声明在函数里，生成tf.constant,如果我们想把变量生成tf.Variable的话
# 首先如果把变量生成为tf.variable的话，他不能在function里面，因为tf.variable必须在function外面去初始化
# 用tf.function标注的函数呢，不能再函数内部声明一个变量，我们只能把变量放到外面
# 讲这个差别是因为，在我们定义神经网络的过程中呢，其实我们用的更多的是这个tf.variable.
# 所以在定义神经网络的过程中呢，我们在转换成function之前，需要把这些变量都全部初始化
# 这是我们讲这个的目的。虽然tf.constant可以放在function内部，但我们在定义神经网络的时候
# 这个tf.constant还是用的比较少的，我们主要还是用tf.variable
var = tf.Variable(0.) 

@tf.function
def add_21():
    return var.assign_add(21) # 给变量加21，因为是变量，不能直接调用加法，要用tf.assign_add函数,相当于+=

print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


In [0]:
#在这个例子里呢，我们会给这个函数加一个输入，由输入计算输出
#这个函数是用来计算立方的
@tf.function
def cube(z):
  return tf.pow(z, 3)

print(cube(tf.constant([1.,2.,3.])))
print(cube(tf.constant([1,2,3])))
#这里呢，输入的constant的类型可以是float,也可以是int32类型的
#那我们可以不可以对输入类型做一个限定
#首先为什么要做限定，做限定的目的就是可以让我输入的数据类型特别明确
#大家知道python是一种弱类型的语言，他的参数呢是没有类别信息的，很容易让大家调错
#如果大家能把这个函数的签名呢，把这个函数里面参数的类型呢，给明确下，那么我们就可以在使用这个函数的时候少犯错了
#这是加类型限制的一个好处
#下面演示如何加类型限制

tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [3]:
#加类型限制呢，主要在tf.function的标注里呢，去把他细化下就可以了
#在这里有个参数叫做input_signature，这个参数呢可以帮我们定义他的输入参数的类型是什么样的
#在这里我们把他限制成int32,首先呢，他的是一个tensorflow的数据类型，其次呢他的类型必须是int32
#然后我们给他定义一个名字x.这样呢，我们就把z的类型限定呢，给做好了
#即可以使用input_signature来限制函数的输入类型，以防止调错
#input_signature还有一个作用，就是我一个函数有了这个input_signature之后
#在tensorflow里才可以保存为这个saved.model
#然后在保存为saved.model过程中，我们需要一个get_concrete_function的函数来把一个tf.function标注的普通的python函数呢
#给变成这种带有图定义的函数
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)

#这个因为类型不对，就会报错，用try catch包住
try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)
    
print(cube(tf.constant([1, 2, 3])))
#这就是如何在tf.function给我们函数做限定

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [4]:
#即可以使用input_signature来限制函数的输入类型，以防止调错
#input_signature还有一个作用，就是我一个函数有了这个input_signature之后
#在tensorflow里才可以保存为这个saved.model
#然后在保存为saved.model过程中，我们需要一个get_concrete_function的函数来把一个tf.function标注的普通的python函数呢
#给变成这种带有图定义的函数
# @tf.function py func -> tf graph
# get_concrete_function -> 它可以给这种由add tf.function来标注的python function 给他加上input signature
#             -> 从而让这个python函数变成要给可以保存的tensorflow图结构。即SavedModel
# 下面演示如何使用get_concrete_function
# 调用只需要在函数名上调用.get_concrete_function就可以了
# 一个函数名呢，它由tf.function来标注之后呢，他就有了其他的属性。get_concrete_function就是其中之一
# get_concrete_function的参数就是输入数据的类型，跟我们定义input_signature是一样的。
# 这样我们就得到一个可以保存的tensorflow function
cube_func_int32 = cube.get_concrete_function(
    tf.TensorSpec([None], tf.int32))
print(cube_func_int32)

In [5]:
#上面的get_concrete_function的参数呢是一个tensorflow类型，除了类型呢，还可以使用一个具体的数传进来，也可以得到一个function的定义
print(cube_func_int32 is cube.get_concrete_function(
    tf.TensorSpec([5], tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(
    tf.constant([1, 2, 3]))) #输入一个常量

#返回都是true 这俩的signature都是一样的

True
True


In [15]:
cube_func_int32.graph #得到这个Concrete对象以后，可以把这个图打印出来

In [7]:
cube_func_int32.graph.get_operations() #图定义里有哪些操作，有四个操作，x, pow/y, pow, Identity,其中第二个就是立方的操作

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [8]:
pow_op = cube_func_int32.graph.get_operations()[2] #把Pow这个操作打印出来
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [9]:
print(list(pow_op.inputs)) #可以把上面pow_op里面的输入由两个x,pow/y
print(list(pow_op.outputs)) #输出就是pow


[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [13]:
cube_func_int32.graph.get_operation_by_name("x") #在给graph里面根据名字x 获取一个operation
#Placeholder用来放输入的，就是我放进去一个输入才能得到输出，在tensorflow1.0里面这个placeholder是一个可以显示定义的东西
#在tensorflow2.0里面呢，我们就不要这个东西了，但是呢，在图定义里依然存在

<tf.Operation 'x' type=Placeholder>

In [16]:
cube_func_int32.graph.get_tensor_by_name("x:0") #x后面会加上一个 :0， 获取tensor，一般会在name后面加上 :0
#我们就会得到x对应的tensor
#然后大家会问，你在这里写的这些字符串x或者x:0,这些字符串你是怎么知道的呢
#第一点就是他们是层次的名字，或者是tensor的名字，在很多时候呢，你是可以给tensor设定名字
#这样呢，你肯定知道你设定的名字是什么
#第二点呢，就是你有个粗暴的方法可以去看一下都会有什么样的名字
#粗暴的方法就是我们把graph_def打印出来

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [12]:
#打印graph_def
#在这个graph_de里面呢，有一个node是x,他的op是Placeholder，跟我们上面是对应的上的。
#还有一个node呢 是 Pow/y，这个呢应该是我们输入的幂次那个信息
#然后还有我们去计算立方的那个pow节点
#然后在计算完立方以后，我们还有一个节点叫Identity,输入是什么，输出就是什么，这个一般会作为过渡节点
cube_func_int32.graph.as_graph_def()

#上面内容虽然跟训练模型没关，因为我们训练model,只需要用keras去搭建，然后调用keras里面的fit函数就可以去train了
#上面的函数呢，一般会用在两个地方
#第一个地方呢： 就是你如何去保存模型
#第二个地方呢： 你在保存了模型以后，你如何用你保存的模型载入进来，然后我们在载入模型去做inference的时候可能会用到这些函数
#比如get_operation_by_name  get_tensor_by_name,如果采用c++或者其他语言去写的话，经常会用到

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 175
}